In [ ]:
# Code needed for binder 
import os
os.environ["COVID_DATA_PUBLIC"] = "covid-data-public"

In [ ]:
from libs.datasets.sources.can_pyseir_location_output import CANPyseirLocationOutput
from libs.qa import dataset_summary
from libs.datasets import combined_datasets
from IPython.display import display
from ipywidgets import interact
import ipywidgets as widgets
from libs.datasets.dataset_utils import AggregationLevel
from covidactnow.datapublic.common_fields import CommonFields
import pandas as pd
from libs.datasets.sources import cmdc
from libs.datasets.sources import cds_dataset
from libs.datasets.sources import covid_tracking_source
from libs.datasets.sources import nytimes_dataset
from libs.datasets.sources import jhu_dataset
from libs.datasets.sources import nha_hospitalization
from libs.datasets.sources import texas_hospitalizations
from libs.datasets.timeseries import TimeseriesDataset
pd.options.display.max_rows = 3000
pd.options.display.max_columns = 3000
from IPython.display import HTML


In [ ]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')

In [ ]:
def load_combined_timeseries(timeseries):
    sources = [
        cmdc.CmdcDataSource,
        cds_dataset.CDSDataset,
        jhu_dataset.JHUDataset,
        nha_hospitalization.NevadaHospitalAssociationData,
        texas_hospitalizations.TexasHospitalizations,
        covid_tracking_source.CovidTrackingDataSource,
        nytimes_dataset.NYTimesDataset
    ]
    all_data = []
    for source_cls in sources:
        timeseries_dataset = TimeseriesDataset.build_from_data_source(source_cls.local())
        timeseries_dataset.data["source"] = source_cls.__name__
        all_data.append(timeseries_dataset.data)

    timeseries_data = timeseries.data.copy()
    timeseries_data["source"] = "Combined Data"

    combined_timeseries = TimeseriesDataset(pd.concat([timeseries_data] + all_data))
    return combined_timeseries

def filter_field_data(combined_series, field, state=None, fips=None, after=None, diff=False, before=None):
    columns = [CommonFields.FIPS, CommonFields.DATE] + [field, "source"]
    level = None
    if state and not fips:
        combined_series = combined_series.get_subset(aggregation_level=AggregationLevel.STATE)

    data = combined_series.get_data(state=state, fips=fips, after=after, columns_slice=columns, before=before)
    data = data.set_index(["fips", "date", "source"])
    series = data[field]
    if diff:
        series = series.diff()
    return series.unstack(level=-1)    
    
def filter_timeseries(state=None, fips=None, columns=None, after=None, before=None, diff=False):
    if columns:
        columns = [CommonFields.FIPS, CommonFields.DATE] + columns
    us_timeseries = timeseries
    level = None
    if state and not fips:
        us_timeseries = us_timeseries.get_subset(aggregation_level=AggregationLevel.STATE)
        
    data = us_timeseries.get_data(state=state, fips=fips, columns_slice=columns, after=after).set_index(['fips', 'date'])
    if not diff:
        return data
    
    return data.diff()


In [ ]:
summary = dataset_summary.load_summary()
timeseries = combined_datasets.load_us_timeseries_dataset()
combined_timeseries = load_combined_timeseries(timeseries)

states = [" "] + timeseries.states
columns = timeseries.data.columns
not_included_columns = ['fips', 'date', 'state', 'county', 'country', 'aggregate_level']
columns = [column for column in columns if column not in not_included_columns]

In [ ]:
column_selector = widgets.SelectMultiple(
    options=[column for column in columns if column not in ['fips', 'date']], 
    value=["cases", "deaths", "current_hospitalized", "current_icu", "positive_tests", "negative_tests"]
)

@interact
def display_timeseries(
    state=states, fips="", columns=column_selector, after="2020-07-01", before="", diff=False
):
    if before == "":
        before = None
    if state == " ":
        state = None
    columns = list(columns)
    if not state and not fips:
        return
    data = filter_timeseries(state=state, fips=fips, columns=columns, after=after, diff=diff, before=before)
    display(data)

In [ ]:
@interact
def display_single_field(state=states, fips="", field=columns, after="2020-07-15", before="", diff=False):
    if state == " ":
        state = None
    if before == "":
        before == ""
    if not state and not fips:
        return
    data = filter_field_data(combined_timeseries, field, state=state, fips=fips, after=after, diff=diff, before=before)
    display(data)